<a href="https://colab.research.google.com/github/Susovan88/Machine_Learning/blob/main/Feature_Selection/Hyperparameter_Tuning_And_Data_Lickage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

url = "https://raw.githubusercontent.com/ageron/handson-ml/master/datasets/housing/housing.csv"
df = pd.read_csv(url)
df.head()


,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


In [2]:
df.isnull().sum()

,0
longitude,0
latitude,0
housing_median_age,0
total_rooms,0
total_bedrooms,207
population,0
households,0
median_income,0
median_house_value,0
ocean_proximity,0


In [3]:
df=df.dropna()
df.shape

(20433, 10)

In [4]:
X=df.drop(columns=['longitude','latitude','ocean_proximity','median_house_value'],axis=1)
y=df[['median_house_value']]

X.shape

(20433, 6)

In [5]:
from sklearn.model_selection import LeaveOneOut,cross_val_score,train_test_split
from sklearn.metrics import r2_score,root_mean_squared_error

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=412)

X_train

,housing_median_age,total_rooms,total_bedrooms,population,households,median_income
6471,36.0,1239.0,238.0,717.0,237.0,3.2440
2532,23.0,462.0,97.0,261.0,90.0,2.1705
19553,28.0,1153.0,420.0,1043.0,357.0,1.0801
9933,31.0,1633.0,316.0,944.0,300.0,3.3977
18191,25.0,2213.0,360.0,1066.0,390.0,7.2165
...,...,...,...,...,...,...
74,20.0,835.0,161.0,290.0,133.0,2.4830
13658,7.0,1971.0,403.0,1336.0,423.0,4.5066
20334,11.0,5785.0,1035.0,2760.0,985.0,4.6930
9153,19.0,1285.0,195.0,650.0,193.0,6.0398


In [6]:
from sklearn.linear_model import LinearRegression

Lreg=LinearRegression()

Lreg.fit(X_train,y_train)
y_pred=Lreg.predict(X_test)
print(f"Linear regression r2 ->{r2_score(y_test,y_pred)} , root means square error -> {root_mean_squared_error(y_test,y_pred)}")

Linear regression r2 ->0.5791388891058505 , root means square error -> 75504.5963770102


# K Fold CV

In [7]:
from sklearn.model_selection import KFold

kfold=KFold(n_splits=10,random_state=42,shuffle=True)
reg=LinearRegression()

scores=cross_val_score(reg,X_train,y_train,cv=kfold,scoring='r2')

print("r2_scores-> ",scores)
print('mean r2 score -> ',scores.mean())

r2_scores->  [0.58639525 0.58389346 0.57040808 0.57059516 0.55802053 0.6045596
 0.52360129 0.53736727 0.53441389 0.5512779 ]
mean r2 score ->  0.5620532433190131


## Stratified K Fold

In [8]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split,StratifiedKFold
from sklearn.preprocessing import StandardScaler

data= pd.read_csv('https://raw.githubusercontent.com/npradaschnor/Pima-Indians-Diabetes-Dataset/master/diabetes.csv')

X=data.drop('Outcome',axis=1)
y=data['Outcome']

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)
std_scaler=StandardScaler()

X_train=std_scaler.fit_transform(X_train)
X_test=std_scaler.transform(X_test)

L_reg=LogisticRegression(max_iter=100)
SKfold=StratifiedKFold(n_splits=10,shuffle=True,random_state=42)

scores=cross_val_score(L_reg,X_train,y_train,cv=SKfold,scoring='accuracy')

print("Accuracies for each fold: ", scores)
print("Mean accuracy across all folds: ", scores.mean())


Accuracies for each fold:  [0.79032258 0.74193548 0.83870968 0.70967742 0.72131148 0.7704918
 0.75409836 0.72131148 0.85245902 0.68852459]
Mean accuracy across all folds:  0.7588841882601798


# Leave One Out

In [9]:
# import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import LeaveOneOut, cross_val_score

# Load the Boston Housing dataset
df = pd.read_csv('https://raw.githubusercontent.com/selva86/datasets/master/BostonHousing.csv')
X = df.iloc[:,0:-1]
y = df.iloc[:,-1]

# Create a linear regression model
model = LinearRegression()

# Create a LeaveOneOut cross-validator
loo = LeaveOneOut()

scores = cross_val_score(model, X, y, cv=loo, scoring='neg_mean_squared_error')

mse_scores = -scores  # Invert the sign of the scores

# Print the mean MSE over all LOOCV iterations
print("Mean MSE:", mse_scores.mean())

Mean MSE: 23.72574551947613


## Hyperparameter Tuning

In [10]:
df = pd.read_csv('https://raw.githubusercontent.com/selva86/datasets/master/BostonHousing.csv')
X = df.iloc[:,:-1]
y = df.iloc[:,-1]

In [11]:
from sklearn.model_selection import cross_val_score,KFold
from sklearn.neighbors import KNeighborsRegressor

knn=KNeighborsRegressor()
kfold=KFold(n_splits=10,shuffle=True,random_state=42)

scores=cross_val_score(knn,X,y,cv=kfold,scoring='r2')
print("Mean MSE:", scores.mean())
print(scores)

Mean MSE: 0.5349410296042879
[0.55261561 0.67525226 0.57929155 0.70341976 0.46622585 0.27384763
 0.63339056 0.55445637 0.60653071 0.30437999]


# GridSearch CV

In [12]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'n_neighbors':[1,3,5,7,10,12,15,17,20],
    'weights':['uniform','distance'],
    'algorithm':['ball_tree', 'kd_tree', 'brute'],
    'p':[1,2]
}

knn=KNeighborsRegressor()

gsCV=GridSearchCV(knn,param_grid,scoring='r2')

gsCV.fit(X,y)


GridSearchCV(estimator=KNeighborsRegressor(),
             param_grid={'algorithm': ['ball_tree', 'kd_tree', 'brute'],
                         'n_neighbors': [1, 3, 5, 7, 10, 12, 15, 17, 20],
                         'p': [1, 2], 'weights': ['uniform', 'distance']},
             scoring='r2')

In [13]:
gsCV.best_params_

{'algorithm': 'ball_tree', 'n_neighbors': 3, 'p': 1, 'weights': 'uniform'}

In [14]:
gsCV.best_score_

np.float64(-0.19870846489933208)

## RandomSearchCV

In [17]:
from sklearn.model_selection import RandomizedSearchCV

knn=KNeighborsRegressor()

random_cv=RandomizedSearchCV(knn,param_grid,n_iter=100,scoring='r2')
random_cv.fit(X,y)

random_cv.best_score_

np.float64(-0.19870846489933208)

In [18]:
random_cv.best_params_

{'weights': 'uniform', 'p': 1, 'n_neighbors': 3, 'algorithm': 'kd_tree'}